# requirement

> pip install tqdm pandas

# tf.data

In [3]:
import tensorflow as tf
from tqdm import tqdm
from tqdm import trange
import os
import pandas as pd

## 실제로 데이터가 주어진 경우  해결방법 예시
강아지 고양이가 있고 이를 구별하기 

먼저 inpu pipline을 구성해야함 

두가지 방법이 있음 .

There are two distinct ways to create a dataset: <br>

1. A data source constructs a Dataset from data stored in memory or in one or more files.
2. A data transformation constructs a dataset from one or more tf.data.Dataset objects.

In [6]:
def test_speed(ds, test_size=5000, warmup=None, print_top_n=0):
    it = ds.make_one_shot_iterator()
    next_dp = it.get_next()
    try:
        with tf.Session() as sess:
            if warmup:
                for _ in trange(warmup):
                    sess.run(next_dp)

            with tqdm(total=test_size, leave=True, smoothing=0.2) as pbar:
                for _ in range(test_size):
                    if _ < print_top_n:
                        dp = sess.run(next_dp)
                        print('shape: {}'.format([item.shape for item in dp if hasattr(item, 'shape')]))

                        print('data: {}'.format(dp))
                    else:
                        sess.run(next_dp)
                    pbar.update()
    except tf.errors.OutOfRangeError as e:
        print('OutOfRange')

- 아래의  프리프로세싱 작업을 직접 코딩하는 작업이 매우 복잡함 <br>
강사님은 prebatch from generate만 사용한다고 함.

In [9]:
IMAGE_SIZE = 128
BATCH_SIZE = 128

def train_input_fn():
    df = pd.read_csv('./dogs/labels_0.csv',comment='#',names=['filename','label'])
    # os.path.join을 이용해서 경로를 하나로 합쳐줌
    filenames = [os.path.join('./dogs',filename) for filename in df['filename'].values]
    # dataset을 가저오는 역할 
    ds = tf.data.Dataset.from_tensor_slices((filenames,df['label'].values))
    # ds.repeat() 안에 숫자를 넣어주면 반복 횟수 지정가능 안넣는경우 끝까지 반복 수행
    ds = ds.repeat().shuffle(300)
    
    def preprocessing(filename, label):
        file = tf.read_file(filename)
        # 이미지 디코딩 
        image = tf.image.decode_image(file, channels=3)
        # shape을 모르기 때문에 따로 지정해줘야함.
        image.set_shape([None, None, 3])
        # 0~1사이의 이미지 nomalization을 함 
        image = tf.image.per_image_standardization(image)
        # resize -> 이미지 사이즈가 다 다르기 때문에 변경 필요 
        image = tf.image.resize_images(image, [IMAGE_SIZE, IMAGE_SIZE])
        # label 은 강아지 고양이 둘중하나임으로 2개의 크기의 one-hot encoding을 수행
        return (image, tf.one_hot(label,2))
    # 프리프로세싱 하는경우 주로 map 함수를 사용한다.
    # ex) 데이터 구조가  사진경로, int  로 되어있는걸 -> 사진형태 , one hot encoding으로 변경 
    ds = ds.map(preprocessing, num_parallel_calls=2)
    # 이미지의 shae이 같지 않으면 배치가 되지 않음  batch 하기이전에 image resize필수 
    ds = ds.batch(BATCH_SIZE)
    #ds = ds.prefetch(10)
    return ds

In [8]:
ds = train_input_fn()
test_speed(ds, test_size=10, warmup=10, print_top_n=1)

FileNotFoundError: [Errno 2] File ./dogs/labels_0.csv does not exist: './dogs/labels_0.csv'